In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [29]:
ratings=pd.read_csv("BX-Book-Ratings.csv", nrows=10000)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  10000 non-null  int64 
 1   isbn     10000 non-null  object
 2   rating   10000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 234.5+ KB


In [30]:
ratings=ratings.drop(['rating'],axis=1)


In [31]:
ratings['user_id']=ratings.user_id.astype('str')

In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  10000 non-null  object
 1   isbn     10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [33]:
import chardet
with open("BX-Books.csv", 'rb') as rawdata:
    book = chardet.detect(rawdata.read(100000))
book

{'encoding': 'ISO-8859-1', 'confidence': 0.717981570018507, 'language': ''}

In [34]:
books=pd.read_csv("BX-Books.csv", encoding='ISO-8859-1')


c:\users\tejshree reddy\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
Merged=(ratings.merge(books, left_on='isbn', right_on='isbn'))

In [36]:
Merged

,user_id,isbn,book_title,book_author,year_of_publication,publisher
0,276725,034545104X,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276726,155061224,Rites of Passage,Judith Rae,2001,Heinle
2,276727,446520802,The Notebook,Nicholas Sparks,1996,Warner Books
3,278418,446520802,The Notebook,Nicholas Sparks,1996,Warner Books
4,276729,052165615X,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
...,...,...,...,...,...,...
8696,243,385720106,A Map of the World,Jane Hamilton,1999,Anchor Books/Doubleday
8697,243,425092917,The Accidental Tourist,Anne Tyler,1994,Berkley Publishing Group
8698,243,425098834,If Morning Ever Comes,Anne Tyler,1983,Berkley Publishing Group
8699,243,425163407,Unnatural Exposure,Patricia Daniels Cornwell,1998,Berkley Publishing Group


In [37]:
df=Merged.drop(['isbn','book_author','year_of_publication','publisher'],axis=1)

In [38]:
df

,user_id,book_title
0,276725,Flesh Tones: A Novel
1,276726,Rites of Passage
2,276727,The Notebook
3,278418,The Notebook
4,276729,Help!: Level 1
...,...,...
8696,243,A Map of the World
8697,243,The Accidental Tourist
8698,243,If Morning Ever Comes
8699,243,Unnatural Exposure


In [40]:
# Grat=ratings.groupby('user_id')

In [42]:
# Grat.get_group('97')

In [39]:
isbn_in_list=df.groupby('user_id')['book_title'].apply(list)


In [40]:
isbn=isbn_in_list.reset_index(drop=True)


In [41]:
isbn_list=isbn.tolist()

In [42]:
from apyori import apriori
rules = apriori(transactions = isbn_list, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2)

In [43]:
results = list(rules)

In [44]:
results

[RelationRecord(items=frozenset({"Songs in Ordinary Time (Oprah's Book Club (Paperback))", 'A Lesson Before Dying (Vintage Contemporaries (Paperback))'}), support=0.0036231884057971015, ordered_statistics=[OrderedStatistic(items_base=frozenset({'A Lesson Before Dying (Vintage Contemporaries (Paperback))'}), items_add=frozenset({"Songs in Ordinary Time (Oprah's Book Club (Paperback))"}), confidence=1.0, lift=276.0), OrderedStatistic(items_base=frozenset({"Songs in Ordinary Time (Oprah's Book Club (Paperback))"}), items_add=frozenset({'A Lesson Before Dying (Vintage Contemporaries (Paperback))'}), confidence=1.0, lift=276.0)]),
 RelationRecord(items=frozenset({'Breathing Lessons', 'The Brethren'}), support=0.0036231884057971015, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Breathing Lessons'}), items_add=frozenset({'The Brethren'}), confidence=0.6, lift=124.2), OrderedStatistic(items_base=frozenset({'The Brethren'}), items_add=frozenset({'Breathing Lessons'}), confidence=0

In [45]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])




In [46]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,A Lesson Before Dying (Vintage Contemporaries ...,Songs in Ordinary Time (Oprah's Book Club (Pap...,0.003623,1.000000,276.000000
1,Breathing Lessons,The Brethren,0.003623,0.600000,124.200000
2,Deadlock (V.I. Warshawski Novels (Paperback)),The Street Lawyer,0.003623,1.000000,82.800000
3,Lucky : A Memoir,The Lovely Bones: A Novel,0.003623,1.000000,92.000000
4,Mystic River,Wild Animus,0.003623,0.428571,20.873950
5,She's Come Undone (Oprah's Book Club (Paperback)),Wild Animus,0.003623,0.600000,29.223529
6,Summer Sisters,The Brethren,0.003623,0.600000,124.200000
7,The Da Vinci Code,The Lovely Bones: A Novel,0.003623,0.750000,69.000000
8,The Indian in the Cupboard,The Return of the Indian (Indian in the Cupboard),0.003623,0.750000,207.000000
9,The Street Lawyer,Vanished,0.003623,0.300000,62.100000


In [47]:
final_result=resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

In [48]:
final_result

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,A Lesson Before Dying (Vintage Contemporaries ...,Songs in Ordinary Time (Oprah's Book Club (Pap...,0.003623,1.000000,276.000000
8,The Indian in the Cupboard,The Return of the Indian (Indian in the Cupboard),0.003623,0.750000,207.000000
1,Breathing Lessons,The Brethren,0.003623,0.600000,124.200000
6,Summer Sisters,The Brethren,0.003623,0.600000,124.200000
3,Lucky : A Memoir,The Lovely Bones: A Novel,0.003623,1.000000,92.000000
2,Deadlock (V.I. Warshawski Novels (Paperback)),The Street Lawyer,0.003623,1.000000,82.800000
7,The Da Vinci Code,The Lovely Bones: A Novel,0.003623,0.750000,69.000000
9,The Street Lawyer,Vanished,0.003623,0.300000,62.100000
5,She's Come Undone (Oprah's Book Club (Paperback)),Wild Animus,0.003623,0.600000,29.223529
4,Mystic River,Wild Animus,0.003623,0.428571,20.873950
